# 1. Include the Required Header files

In [ ]:
# Import the necessary header files
import face_recognition
import cv2
import numpy as np
import csv
import sys
# import import_ipynb
import pickle
import os

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# 2. Add the user faces 

In [ ]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# User database of face images.
user_names = []
face_encodings = []
gmail = []

first = True

# Open a video window and capture the face to be added
while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Get all the faces locations in the frame.
    face_locations = face_recognition.face_locations(rgb_small_frame)
    
    # Get encoding for all the faces in the frame.
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
    # Check if a frame contains the faces.
    if face_locations != []:
            
        # Skip first 100 frames for user to set up his face.
        if first:
            
            # Skipping 100 frames
            for i in range(1,50):
                # Grab a single frame of video
                ret, frame = video_capture.read()
                cv2.waitKey(1)
                # Display the resulting image
                cv2.imshow('Video3', frame)

            first = False
            continue
            
        for (top, right, bottom, left) in face_locations:
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4
            
            temp_frame = frame
            
            # Draw a box around the face
            cv2.rectangle(temp_frame, (left, top), (right, bottom), (0, 0, 255), 2)
            
            # Display the resulting image
            cv2.imshow('Video3', temp_frame)
    
        
            cv2.waitKey(0)
            
            # Getting the user names and saving it.
            print('Enter Name :- ')
            user_name = input()
            user_names.append(user_name)
            print('Enter Email :- ')
            gmail_userid = input()
            print('Enter Password :- ')
            gmail_password = input()
            gmail.append(gmail_userid)
            gmail.append(gmail_password)
            cv2.waitKey(1)
        
        # Display the resulting image
        cv2.imshow('Video3', frame)
    
        # Wait for some time for the user to see the image.
        cv2.waitKey(3000)
        
        # Release handle to the webcam
        video_capture.release()
        cv2.destroyAllWindows()
        break
    else: 
        first = True
    
    # Display the resulting image
    cv2.imshow('Video3', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
if video_capture is not None:
    video_capture.release()
    cv2.destroyAllWindows()

# with open('data.csv', 'a') as csvFile:
#     reader = csv.reader(csvFile)
#     for row in reader:
#         if row[0] == user_names[0]:
#             print('User already exists')
#             write = False

In [ ]:
# #GUI features bypassed
# d={"gmail":gmail,"facebook":["aditya020998@gmail.com","Crazy98"]}
# file=open(user_names[0] + ".pickle","wb")
# pickle.dump(d,file)
# file.close()

# 3. Save the data

In [ ]:
data = []
data.append(user_names[0])

print(data)

write = True

print(data)
            
if write:
    w = csv.writer(open("data.csv", "a"))
    w.writerow(data)
    outfile = data[0]+'.npy'
    np.save(outfile, face_encodings[0])

# 4. Password manager

In [ ]:
def details(website,user,passw):
    site=website
    usernameStr = user
    passwordStr = passw
    return [site,usernameStr,passwordStr]
def sites(site):
    browser = webdriver.Chrome("/usr/lib/chrome-browser/chromedriver")
    if site=="gmail":
        browser.get(('https://accounts.google.com/ServiceLogin?'
             'service=mail&continue=https://mail.google'
             '.com/mail/#identifier'))
    elif site=="facebook":
        browser.get(('https://www.facebook.com/login/device-based/regular/login/?login_attempt=1&lwv=110/'))
    return browser
def run(s,u,p):
    res=details(s,u,p)
    usernameStr=res[1]
    browser=sites(res[0])
    # fill in username and hit the next button
    if res[0]=="gmail":
        username = browser.find_element_by_id('identifierId')
        username.send_keys(usernameStr)
        nextButton = browser.find_element_by_id('identifierNext')
        nextButton.click()

    elif res[0]=="facebook":
        username = browser.find_element_by_id('email')
        username.send_keys(usernameStr)
    time.sleep(1)
    password(res,browser)
    
def password(res,browser):
    passwordStr=res[2]
    if res[0]=="gmail":
        password = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.NAME, 'password')))
        password.send_keys(passwordStr)

        signInButton = browser.find_element_by_id('passwordNext')
        signInButton.click()

    elif res[0]=="facebook":
        password = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.NAME, 'pass')))
        password.send_keys(passwordStr)
        
        signInButton = browser.find_element_by_id('loginbutton')
        signInButton.click()

In [ ]:
def password_auto(face_names):
    if len(face_names)>1:
        for i in range(len(face_names)):
            print(i+1," ",face_names[i])
            choice=int(input("Enter index:"))
            print("User: ",face_names[choice-1])
    else:
        choice=1
        print("User: ",face_names[0])
        
    site=input("Enter site: gmail/facebook ")
    file=open(face_names[choice-1]+".pickle","rb")
    d_lo=pickle.load(file)
    usr_pas=d_lo[site]
    u=usr_pas[0]
    p=usr_pas[1]
    run(site,u,p)

# 5. Detect the face

In [ ]:
# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)
known_face_encodings = []
known_face_names = []

with open('data.csv', 'r') as csvFile:
    reader = csv.reader(csvFile)
#     next(reader)
    for row in reader:
        known_face_names.append(row[0])

for name in known_face_names:
    known_face_encodings.append(np.load(name+'.npy'))

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()
    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding, 0.6)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()
password_auto(face_names)

In [ ]:
# #GUI features bypassed
# d={"gmail":["aditya020998@gmail.com","Sharingan98"],"facebook":["aditya020998@gmail.com","Crazy98"]}
# file=open("aditya.pickle","wb")
# pickle.dump(d,file)
# file.close()

In [ ]:
print(face_names)